# Advanced Deep Learning for NLP - Text based adventure game RL


WIP: Links not assigned yet

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

## Google Colab stuff

In [ ]:
%%capture
!pip install tf-agents
!pip install textworld

from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/DeepLearningNLP/")

Or alternatively (for future, when project not private anymore):

In [ ]:
!git clone https://github.com/

Cloning into 'github.com'...
fatal: repository 'https://github.com/' not found


## Imports

In [18]:
from tests import test_environment_creation
import train

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test environment

In [ ]:
test_environment_creation()

# Train

In [21]:
hp_dict = {
  "num_iterations": 1000,
  "learning_rate": 1e-3,
  "initial_collect_steps": 100,
  "collect_steps_per_iteration": 1,
  "replay_buffer_max_length": 100000,
  "batch_size": 64,
  "log_interval": 5,
  "num_eval_episodes": 3,
  "eval_interval": 100,
}

In [ ]:
eval_scores = train.main(plot_avg_ret=True, debug=False, **hp_dict)
print(eval_scores)

## Hyper parameter search

In [ ]:
!pip install optuna
import optuna

In [ ]:
def objective(trial):
    """"""

    hp_dict = {
      "num_iterations": 1000,
      "learning_rate": trial.suggest_loguniform("lr", 1e-4, 1e-2),
      "initial_collect_steps": 100,
      "collect_steps_per_iteration": 1,
      "replay_buffer_max_length": 100000,
      "batch_size": trial.suggest_int("batch_size", 16, 256),
      "log_interval": 50,
      "num_eval_episodes": 2,
      "eval_interval": 250,
    }

    eval_scores = train.main(plot_avg_ret=False, debug=False, **hp_dict)
    eval_scores = np.delete(eval_scores, 0)

    return max(eval_scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)
print(study.best_trial)